In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import pandas as pd

In [2]:
trait_name = "python_client_demo.relations_v1"
layer_name = "relations"

In [4]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

In [ ]:
# TODO fix the trait deffinition
okc.upsert_trait(okapiclient, TraitDefinition(trait_name, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_TEXT),
        TraitRelationDefinition("id", "has_host")
      ]))

Fetch all trait entities from the specified layer

In [5]:
all_traitentities = okc.get_all_traitentities_list(okapiclient, trait_name=trait_name, layers=[layer_name])
all_traitentities

[{'ciid': 'c66142fb-e56b-48ea-8165-edbe7cb2a7bf', 'id': 'A'},
 {'ciid': '35f2f0f9-592e-45e4-b388-55e68f0f3951', 'id': 'B'}]

Fetch all trait relations from the specified layer, in the result we will have the related ci ids

In [6]:
all_trait_relations = okc.get_trait_relation(okapiclient, trait_name=trait_name, relation_name="has_host", layers=[layer_name])
all_trait_relations

[{'base_ciid': 'c66142fb-e56b-48ea-8165-edbe7cb2a7bf',
  'related_ciids': ['35f2f0f9-592e-45e4-b388-55e68f0f3951']},
 {'base_ciid': '35f2f0f9-592e-45e4-b388-55e68f0f3951', 'related_ciids': []}]

Now get all trait entities with realtions, in the result each dictionary will include also a key with a list of all related cis

In [7]:
all_traitentities_with_relations = okc.get_traitentities_with_relations_list(okapiclient, trait_name=trait_name, relation_name="has_host", layers=[layer_name])
all_traitentities_with_relations

[{'ciid': 'c66142fb-e56b-48ea-8165-edbe7cb2a7bf',
  'id': 'A',
  'related_ciis': [{'ciid': '35f2f0f9-592e-45e4-b388-55e68f0f3951',
    'id': 'B'}]},
 {'ciid': '35f2f0f9-592e-45e4-b388-55e68f0f3951',
  'id': 'B',
  'related_ciis': []}]

The result could easily be converted into a pandas dataframe

In [10]:
all_traitentities_with_relations_df = pd.DataFrame(all_traitentities_with_relations)
all_traitentities_with_relations_df

,ciid,id,related_ciis
0,c66142fb-e56b-48ea-8165-edbe7cb2a7bf,A,[{'ciid': '35f2f0f9-592e-45e4-b388-55e68f0f395...
1,35f2f0f9-592e-45e4-b388-55e68f0f3951,B,[]


In [9]:
all_traitentities_with_relations_df['related_ciis'][0]

[{'ciid': '35f2f0f9-592e-45e4-b388-55e68f0f3951', 'id': 'B'}]

TODO add examples on how to merged data using pandas dataframe